https://learn.microsoft.com/en-us/azure/databricks/machine-learning/feature-store/online-feature-store

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient
fe = FeatureEngineeringClient()

In [0]:
CATALOG = "hub"      # Replace with your catalog name
SCHEMA = "default"    # Replace with your schema name

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType
from pyspark.sql import functions as F

NAME = f"{CATALOG}.{SCHEMA}.offline_feature_table"

spark.sql(f"""
CREATE OR REPLACE TABLE {NAME} (
  id INT NOT NULL PRIMARY KEY,
  f1 FLOAT NOT NULL
) USING DELTA
""")

import random

schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("f1", FloatType(), False)
])

data = [
    (i, float(i + random.randint(23, 203)) / max(1, random.randint(1, 45)) * random.random())
    for i in range(1000)
]

df = spark.createDataFrame(data, schema=schema)
df.write.mode("append").saveAsTable(NAME)


In [0]:
from databricks.feature_engineering import FeatureLookup

# The `FeatureSpec` can be accessed in Unity Catalog as a function.
# `FeatureSpec`s can be used to create training sets or feature serving endpoints.
fe.create_feature_spec(
  name = f"{CATALOG}.{SCHEMA}.feature_spec",
  features=[
    FeatureLookup(
      table_name=f"{CATALOG}.{SCHEMA}.offline_feature_table",
      lookup_key="id",
    ),
  ],
)

In [0]:
# Create an online store with specified capacity
fe.create_online_store(
    name="my-online-store",
    capacity="CU_1"  # Valid options: "CU_1", "CU_2", "CU_4", "CU_8"
)

In [0]:
# Get information about an existing online store
online_store = fe.get_online_store(name="my-online-store")
if online_store:
    print(f"Store: {online_store.name}, State: {online_store.state}, Capacity: {online_store.capacity}")

In [0]:
spark.sql(f"ALTER TABLE {CATALOG}.{SCHEMA}.offline_feature_table SET TBLPROPERTIES (delta.enableChangeDataFeed = true)")

In [0]:
# Publish the feature table to the online store
fe.publish_table(
    online_store=online_store,
    source_table_name=f"{CATALOG}.{SCHEMA}.offline_feature_table",
    online_table_name=f"{CATALOG}.{SCHEMA}.online_feature_table"
)

In [0]:
from databricks.feature_engineering.entities.feature_serving_endpoint import (
  ServedEntity,
  EndpointCoreConfig,
)

fe.create_feature_serving_endpoint(
  name="my-feature-serving-endpoint",
  config=EndpointCoreConfig(
    served_entities=ServedEntity(
      feature_spec_name=f"{CATALOG}.{SCHEMA}.feature_spec",
        workload_size="Small",
        scale_to_zero_enabled=True,
        instance_profile_arn=None,
    )
  )
)

In [0]:
endpoint = fe.get_feature_serving_endpoint(name="my-feature-serving-endpoint")
print(endpoint)

In [0]:
import mlflow.deployments

client = mlflow.deployments.get_deploy_client("databricks")

response = client.predict(
    endpoint="my-feature-serving-endpoint",
    inputs={
        "dataframe_records": [
            {"id": 1},
            {"id": 7},
            {"id": 12345},
        ]
    },
)
print(response)

In [0]:
fe.delete_feature_serving_endpoint(name="my-feature-serving-endpoint")
fe.delete_online_store(name="my-online-store")
fe.delete_feature_spec(name=f"{CATALOG}.{SCHEMA}.feature_spec")